# Easy Agent Tutorial
This notebook file provide three examples of using LLM based agents with different tool sets.

prequisites:
- Python 3.10+
- Install required packages:
  ```bash
  pip install -r "mcp[cli]" smolagent
  ```

## Task Description

如README所述，该项目应用三种方案，从不同的角度实现了agentic RAG的功能。为了演示，这一次我们将会构建一个信安四大会的查询，来进行感兴趣论文的搜索以及基于题目选择合适的会议进行投稿。

## Data Preparation

In [1]:
# 我们将使用dblp数据集来进行演示。首先下载四大会最近几年的会议论文数据：
import requests
import json
import os

if not os.path.exists("dblp_sec_papers.json"):
    with requests.Session() as sess:
        url = "https://dblp.org/search/publ/api"
        params = {"q": "security", "format": "json"}
        tocs = {
            "ndss": "toc:db/conf/ndss/ndss2025.bht:",
            "sp": "toc:db/conf/sp/sp2025.bht:",
            "ccs": "toc:db/conf/ccs/ccs2025.bht:",
            "usenix": "toc:db/conf/uss/uss2025.bht:",
        }
        papers = []
        for k, v in tocs.items():
            response = sess.get(url, params={"q": v, "h": 1000, "format": "json"})
            data = response.json()
            data = data["result"]["hits"]["hit"]
            papers.extend(data)
        with open(f"dblp_sec_papers.json", "w", encoding="utf-8") as f:
            json.dump(papers, f, ensure_ascii=False, indent=4)
else:
    with open(f"dblp_sec_papers.json", "r", encoding="utf-8") as f:
        papers = json.load(f)
papers = [x["info"] for x in papers]
titles = [f"[{x['venue']} {x['year']}] {x['title']}" for x in papers]

In [5]:
# 创建并保存向量数据库
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv()
if "db" not in globals():
    db = FAISS.from_texts(titles, OpenAIEmbeddings())
db.save_local("faiss_db")

在开始之前，先看一下传统的工具调用大概长啥样，有怎样的优缺点

In [7]:
import json
import logging
import re

import dotenv
import tenacity
import yaml
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from openai import OpenAI
from openai.types import *
from openai.types.chat import *


def raw_toolcall():
    client = OpenAI()
    db = FAISS.load_local(
        "faiss_db",
        OpenAIEmbeddings(),
        allow_dangerous_deserialization=True,
    )
    tools_def = [
        {
            "type": "function",
            "name": "query_db",
            "description": "accept keyword and return related information",
            "parameters": {
                "type": "object",
                "properties": {
                    "kw": {
                        "type": "string",
                        "description": "The keyword to query the database",
                    }
                },
                "required": ["kw"],
            },
        }
    ]

    def real_ask(question: str):
        from openai.types.responses.response_input_param import Message

        input_msgs: list[Message] = [
            {
                "type": "message",
                "role": "system",
                "content": f"You are a helpful research assistant. When given a question, you must first decide if you need to query the database to get relevant information. If so, use the tool 'query_db' with appropriate keywords extracted from the question. After getting the information, provide a comprehensive answer based on both the retrieved information and your own knowledge.",
            },
            {
                "type": "message",
                "role": "user",
                "content": question,
            },
        ]
        resp = client.responses.create(
            model="gpt-5.1",
            tools=tools_def,
            input=input_msgs,
            tool_choice="required",
        )
        print(resp.output)
        for toolcall in resp.output:
            if toolcall.type != "function_call":
                continue
            if toolcall.name == "query_db":
                kw = json.loads(toolcall.arguments)["kw"]
                print(f"Keyword: {kw}")
                docs = []
                for skw in kw.split():
                    if not (skw := skw.strip()):
                        continue
                    print(f"Searching for keyword: {skw}")
                    docs.extend(db.similarity_search(skw, k=30))
                rag_result = "\n".join([doc.page_content for doc in docs])
                input_msgs.append(toolcall)
                input_msgs.append(
                    {
                        "type": "function_call_output",
                        "call_id": toolcall.call_id,
                        "output": str(rag_result),
                    }
                )
                print(f"{kw=} appended.")
        if input_msgs[-1]["type"] == "function_call_output":
            resp = client.responses.create(
                model="gpt-5.1",
                input=input_msgs,
                # tools=tools_def,
                stream=True,
            )
            for chunk in resp:
                if chunk.type == "response.output_text.delta":
                    print(chunk.delta, end="", flush=True)
            print()
        else:
            print(resp.output_text)

    while True:
        inp = input("=> ")
        if not inp.strip():
            break
        real_ask(inp)


raw_toolcall()

[ResponseFunctionToolCall(arguments='{"kw":"blockchain LLM papers"}', call_id='call_ZNk69x2z6uSxuPEyhldVmhRc', name='query_db', type='function_call', id='fc_0edcc91604d6e0a4006938e28cf5c0819282e662d761ad49ea', status='completed')]
Keyword: blockchain LLM papers
Searching for keyword: blockchain
Searching for keyword: LLM
Searching for keyword: papers
kw='blockchain LLM papers' appended.
你这个方向目前还比较前沿，严格意义上“区块链 × LLM”的论文不算多，但已经有几类比较典型的结合方式，可以按“区块链为LLM赋能”和“LLM为区块链赋能”来找文献。我先列代表性论文和关键词，方便你自己去搜（Google Scholar / arXiv / dblp），再给你一个按方向分类的阅读建议。

下面所有英文标题你直接复制去搜索就能找到 PDF。

---

## 一、LLM 为区块链赋能（用 LLM 做智能合约/链上安全/分析）

### 1. 智能合约分析与形式化验证

- **PropertyGPT: LLM-driven Formal Verification of Smart Contracts through Retrieval-Augmented Property Generation**  
  NDSS 2025（网络与分布式系统安全研讨会）  
  关键词：  
  - “LLM-driven Formal Verification of Smart Contracts”  
  - “Retrieval-Augmented Property Generation”  
  核心思路：用 LLM 自动生成合约的安全属性 / 规范，再结合形式化验证工具检查。适合关注“LLM + 智能合约安全”的同学。

- 可一并检索的关键词：  
  - “LLM for smart co

### 传统工具调用的优缺点

- 优点：直接用模型原生的 function/tool 调用协议，链路短、开销低，JSON Schema 参数校验清晰。
- 优点：可以精确控制何时调用工具、使用 `tool_choice` 等参数强制执行，消息格式透明、便于调试和流式输出。
- 优点：依赖少，不绑框架，易于插入到现有服务或与其他编排层组合。
- 缺点：需要手写对话状态管理、工具输入输出拼接，容易出错且样板代码多。
- 缺点：缺少自动规划/多步推理、重试、fallback 等封装能力，复杂流程要自行实现。
- 缺点：与特定模型/协议耦合，换提供方或多模型时需适配；安全性与数据清洗（如反序列化、去重）也要自管。

## SmolAgent::CodeAgent

In [6]:
import yaml
from dotenv import load_dotenv
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from smolagents import CodeAgent, OpenAIServerModel, tool

load_dotenv()

db = FAISS.load_local(
    "faiss_db",
    OpenAIEmbeddings(),
    allow_dangerous_deserialization=True,
)
content = f"You are a helpful research assistant. When given a question, you must first decide if you need to query the database to get relevant information. If so, use the tool 'query_db' with appropriate keywords extracted from the question. After getting the information, provide a comprehensive answer based on both the retrieved information and your own knowledge."


@tool
def query_db(kw: str) -> str:
    """
    accept keyword and return related paper titles

    Args:
        kw (str): The keyword to query the database
    """
    docs = []
    for skw in kw.split():
        if not (skw := skw.strip()):
            continue
        print(f"Searching for keyword: {skw}")
        docs.extend(db.similarity_search(skw, k=10))
    rag_result = "\n".join([doc.page_content for doc in docs])
    return rag_result


agent = CodeAgent(
    model=OpenAIServerModel("gpt-5.1"),
    tools=[query_db],
    # verbosity_level=1,
    stream_outputs=True,
)

# from Gradio_UI import GradioUI

# GradioUI(agent).launch()

print(agent.run(f"{content}\n\n{input("=> ")}"))

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You are a helpful research assistant. When given a question, you must first decide if you need to query the     │
│ database to get relevant information. If so, use the tool 'query_db' with appropriate keywords extracted from   │
│ the question. After getting the information, provide a comprehensive answer based on both the retrieved         │
│ information and your own knowledge.                                                                             │
│                                                                                                                 │
│ 找一下llm安全的论文                                                                                             │
│                                                                                                                 │
╰─ OpenAIModel - gpt-5.1 ─────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output()

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Thought:                                                                                                       
  # The user is asking in Chinese: "找一下llm安全的论文" = "Find some papers on LLM security."                     
  # I should query the database using relevant English keywords like "LLM security", "large language model         
  security", "jailbreak attacks", etc.                                                                             
  # I'll start with a broad keyword "LLM security" to get related paper titles.                                    
                                                                                                                   
  papers_llm_security = query_db(kw="LLM security")                                                                
  print(papers_llm_security)                                                                                       
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Searching for keyword: LLM
Searching for keyword: security
Searching for keyword: security


...

Output()

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Thought:                                                                                                       
  # We have the list of 10 LLM 安全相关论文和一个简单的 summary。                                                  
  # 不再需要执行代码，直接用 final_answer 返回给用户一个精炼的中文总结和论文清单。                                 
                                                                                                                   
  from datetime import datetime                                                                                    
                                                                                                                   
  papers = [                                                                                                       
      "[USENIX Security Symposium 2025] When LLMs Go Online: The Emerging Threat of Web-Enabled LLMs.",            
      "[CCS 2025] Give LLMs a Security Course: Securing Retrieval-Augmented Code Generation via Knowledge          
  Injection.",                                                                                                     
      "[CCS 2025] Can Personal Health Information Be Secured in LLM? Privacy Attack and Defense in the Medical     
  Domain.",                                                                                                        
      "[USENIX Security Symposium 2025] SelfDefend: LLMs Can Defend Themselves against Jailbreaking in a           
  Practical Manner.",                                                                                              
      "[NDSS 2025] LLMPirate: LLMs for Black-box Hardware IP Piracy.",                                             
      "[NDSS 2025] RACONTEUR: A Knowledgeable, Insightful, and Portable LLM-Powered Shell Command Explainer.",     
      "[SP 2025] On the (In)Security of LLM App Stores.",                                                          
      "[USENIX Security Symposium 2025] EchoLLM: LLM-Augmented Acoustic Eavesdropping Attack on Bone Conduction    
  Headphones with mmWave Radar.",                                                                                  
      "[SP 2025] Alleviating the Fear of Losing Alignment in LLM Fine-tuning.",                                    
      "[USENIX Security Symposium 2025] We Have a Package for You! A Comprehensive Analysis of Package             
  Hallucinations by Code Generating LLMs."                                                                         
  ]                                                                                                                
                                                                                                                   
  answer_text = f"""我帮你从数据库里找到了 **LLM 安全方向、2025 年顶会** 的代表性论文（USENIX Security / CCS /     
  NDSS / IEEE S&P），可以直接用这些英文题目去 Google Scholar、arXiv 或对应会议官网搜 PDF：                         
                                                                                                                   
  一、联网 / 平台生态安全                                                                                          
  1. {papers[0]}                                                                                                   
     - When LLMs Go Online: Web-enabled LLM 带来的新型攻击面（联网访问网页、API 之后的安全问题）。                 
  2. {papers[6]}                                                                                                   
     - On the (In)Security of LLM App Stores: 研究 LLM 应用商店 / 插件生态中的安全风险（恶意 App、权限滥用等）。   
                                                                                                                   
  二、Jailbreak 与对齐安全                                                                                         
  3. {papers[3]}                                                                                         

Final answer: 我帮你从数据库里找到了 **LLM 安全方向、2025 年顶会** 的代表性论文（USENIX Security / CCS / NDSS / 
IEEE S&P），可以直接用这些英文题目去 Google Scholar、arXiv 或对应会议官网搜 PDF：

一、联网 / 平台生态安全
1. [USENIX Security Symposium 2025] When LLMs Go Online: The Emerging Threat of Web-Enabled LLMs.
   - When LLMs Go Online: Web-enabled LLM 带来的新型攻击面（联网访问网页、API 之后的安全问题）。
2. [SP 2025] On the (In)Security of LLM App Stores.
   - On the (In)Security of LLM App Stores: 研究 LLM 应用商店 / 插件生态中的安全风险（恶意 App、权限滥用等）。

二、Jailbreak 与对齐安全
3. [USENIX Security Symposium 2025] SelfDefend: LLMs Can Defend Themselves against Jailbreaking in a Practical 
Manner.
   - SelfDefend: LLMs Can Defend Themselves against Jailbreaking in a Practical Manner
   - 关注越狱攻击（jailbreak）的防御，让 LLM 具备自我防护能力。
4. [SP 2025] Alleviating the Fear of Losing Alignment in LLM Fine-tuning.
   - Alleviating the Fear of Losing Alignment in LLM Fine-tuning
   - 研究微调过程中对齐（alignment）丢失的问题，以及如何缓解。

三、隐私与敏感领域（医疗）
5. [CCS 2025] Can Personal Health Information Be Secured in LLM? Privacy Attack and Defense in the Medical Domain.
   - Can Personal Health Information Be Secured in LLM? Privacy Attack and Defense in the Medical Domain
   - 医疗场景下的 LLM 隐私攻击与防御，关注个人健康信息（PHI）的泄露风险。

四、代码安全与供应链
6. [CCS 2025] Give LLMs a Security Course: Securing Retrieval-Augmented Code Generation via Knowledge Injection.
   - Give LLMs a Security Course: Securing Retrieval-Augmented Code Generation via Knowledge Injection
   - RAG + 代码生成场景，通过注入安全知识来减少不安全代码。
7. [USENIX Security Symposium 2025] We Have a Package for You! A Comprehensive Analysis of Package Hallucinations 
by Code Generating LLMs.
   - We Have a Package for You! A Comprehensive Analysis of Package Hallucinations by Code Generating LLMs
   - 分析代码 LLM 产生“虚构依赖包（package hallucination）”带来的供应链安全风险。
8. [NDSS 2025] RACONTEUR: A Knowledgeable, Insightful, and Portable LLM-Powered Shell Command Explainer.
   - RACONTEUR: A Knowledgeable, Insightful, and Portable LLM-Powered Shell Command Explainer
   - LLM 驱动的 shell 命令解释器，可视为安全辅助工具（帮助用户理解可能有风险的命令）。

五、LLM 辅助攻击 / 知识产权安全
9. [NDSS 2025] LLMPirate: LLMs for Black-box Hardware IP Piracy.
   - LLMPirate: LLMs for Black-box Hardware IP Piracy
   - 利用 LLM 辅助进行黑盒硬件 IP 盗取，是“LLM 被攻击者利用”的典型案例。
10. [USENIX Security Symposium 2025] EchoLLM: LLM-Augmented Acoustic Eavesdropping Attack on Bone Conduction 
Headphones with mmWave Radar.
    - EchoLLM: LLM-Augmented Acoustic Eavesdropping Attack on Bone Conduction Headphones with mmWave Radar
    - LLM 参与声学/侧信道窃听攻击，展示 LLM 在攻击链中的增强作用。

如果你告诉我你更想做哪一块（比如「越狱防御」「RAG + 安全」「医疗隐私」「代码安全」），我可以在这 10 
篇的基础上，再帮你补充 2023–2024 年比较经典的 LLM 安全论文，顺便给你一条适合作为开题/综述的阅读路线。

[Step 5: Duration 11.28 seconds| Input tokens: 35,967 | Output tokens: 9,134]

我帮你从数据库里找到了 **LLM 安全方向、2025 年顶会** 的代表性论文（USENIX Security / CCS / NDSS / IEEE S&P），可以直接用这些英文题目去 Google Scholar、arXiv 或对应会议官网搜 PDF：

一、联网 / 平台生态安全
1. [USENIX Security Symposium 2025] When LLMs Go Online: The Emerging Threat of Web-Enabled LLMs.
   - When LLMs Go Online: Web-enabled LLM 带来的新型攻击面（联网访问网页、API 之后的安全问题）。
2. [SP 2025] On the (In)Security of LLM App Stores.
   - On the (In)Security of LLM App Stores: 研究 LLM 应用商店 / 插件生态中的安全风险（恶意 App、权限滥用等）。

二、Jailbreak 与对齐安全
3. [USENIX Security Symposium 2025] SelfDefend: LLMs Can Defend Themselves against Jailbreaking in a Practical Manner.
   - SelfDefend: LLMs Can Defend Themselves against Jailbreaking in a Practical Manner
   - 关注越狱攻击（jailbreak）的防御，让 LLM 具备自我防护能力。
4. [SP 2025] Alleviating the Fear of Losing Alignment in LLM Fine-tuning.
   - Alleviating the Fear of Losing Alignment in LLM Fine-tuning
   - 研究微调过程中对齐（alignment）丢失的问题，以及如何缓解。

三、隐私与敏感领域（医疗）
5. [CCS 2025] Can Personal Health Information Be Secured in LLM? Privacy Attack and Defen

### 方案说明：SmolAgent::CodeAgent
- 核心循环：强制每步遵循 “Thought → Code → Observation”，并用 `{{code_block_opening_tag}}`/`{{code_block_closing_tag}}` 包裹代码。中间信息需用 `print` 显式输出，下一步作为 Observation 读取。最终答案必须调用 `final_answer` 工具收束。
- 规划前置：系统 prompt 要求先做 facts survey（已知/待查/待推导）再产出高层计划；若重试或中断，使用 update_plan 模板复盘并继续，保证稳态迭代。
- 工具与调用规范：可用工具（Python 函数）被注入环境，必须按显式参数名调用，禁止 dict 传参，禁止使用未定义变量，也禁止变量名与工具同名；对无结构化输出的工具，提示分步打印避免链式依赖。
- 安全与约束：限制可导入模块白名单；提醒不要链过多非结构化结果，必要时拆分步；保持会话状态，可多轮复用已导入模块和已定义变量。
- 角色定位：将代理设为“能用代码解决任意任务的专家助手”，鼓励用工具+代码完成 RAG 检索、加工与总结；prompt 示例覆盖算术、检索、多模态 QA 等，教会模型如何 Thought/Code/Observation 协同。
- 输出与流式：示例强调逐步打印中间结果、最后用 `final_answer` 汇总；若有 stream 开启，能边执行边输出。
### 优点
- 自带规划、分步执行与错误恢复模板，减少裸 toolcall 的样板与状态管理成本。
- Thought/Code/Observation 将思路与代码解耦，便于调试审计；跨步保留状态，适合多轮 RAG。
- 明确的调用规范（参数名、变量定义、模块白名单）降低调用错误，提升可重复性与安全性。
- 扩展性好：添加/替换工具即可扩展能力，主逻辑无需重写。
### 缺点
- Prompt 体积大、指令冗长，占用上下文并增加生成时延，相比直接 toolcall 成本更高。
- 控制粒度不及手写状态机，精确流式或严格路径需额外封装。
- 依赖工具实现与输出结构；非结构化结果仍需手动清洗，工具安全需自管。
- 简单任务时，完整 Thought/Code/Observation 纪律会带来不必要的步骤和延迟。